### Problem definition

Το dataset 'Severe Weather Data Inventory (SWDI)' του National Oceanic Atmospheric Administration (NOAA) ειναι μια βάση δεδομένων για καιρικά φαινόμενα με έντονη δριμύτητα. Εκεί καταγράφονται πληροφορίες για την εμφάνιση έντονων καιρικών φαινομένων από το 1950 μέχρι σήμερα. Οι πληροφορίες σε γενικές γραμμές αφορούν το είδος του φαινομένου (πχ τυφώνας, πλημμύρα), την τοποθεσία και τον αντίκτυπο σε υλικές ζημιές και στους ανθρώπους. Θα το δούμε πιο λεπτομερώς στη συνέχεια.

##### Data Preprocessing

Εισάγουμε τις βιβλιοθήκες και τα modules που θα χρειαστούμε.

In [1]:
import os
import csv
import math
import time
import datetime
import matplotlib
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
import pandas as pd
from pandasql import sqldf
import seaborn as sns
from sklearn.preprocessing import Normalizer
import copy

Διαβάζουμε το csv αρχείο του dataset σε ένα pandas' DataFrame.

In [2]:
df = pd.read_csv('./data.csv',low_memory=False)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1639004 entries, 0 to 1639003
Data columns (total 34 columns):
Unnamed: 0            1639004 non-null int64
episode_id            1406765 non-null float64
event_id              1639004 non-null int64
state                 1639003 non-null object
state_fips_code       1639003 non-null float64
event_type            1639004 non-null object
cz_type               1639004 non-null object
cz_fips_code          1639004 non-null int64
cz_name               1637447 non-null object
wfo                   1513431 non-null object
event_begin_time      1639004 non-null object
event_timezone        1639004 non-null object
event_end_time        1639004 non-null object
injuries_direct       1639004 non-null int64
injuries_indirect     1639004 non-null int64
deaths_direct         1639004 non-null int64
deaths_indirect       1639004 non-null int64
damage_property       1078595 non-null float64
damage_crops          961257 non-null float64
source           

Από τις πληροφορίες του dataset, βλέπουμε ότι έχουμε 1639004 entries και 34 columns , που αντιστοιχούν στις εξής πληροφορίες:

0. index


1. episode_id:           ID assigned by NWS to denote the storm episode; links the event details file with the information within location file.


2. event_id:             ID assigned by NWS to note a single, small part that goes into a specific storm episode; links the storm episode between the three files downloaded from SPC’s website.


3. state:                The full text state name where the event occurred.


4. state_fips_code:      Unique FIPS code identifier assigned to each state. State names and their corresponding FIPS codes are available as a BigQuery Public Dataset: bigquery-public-data.census_fips_codes.states_2016  The geographic polygons that define the perimeter of each state are available as a BigQuery Public Dataset: bigquery-public-data.geo_us_boundaries.us_states.


5. event_type:           The only events permitted in Storm Data are listed in Table 1 of Section 2.1.1 of NWS Directive 10-1605 at http://www.nws.noaa.gov/directives/sym/pd01016005curr.pdf. The chosen event type is the one that most accurately describes the meteorological event leading to fatalities, injuries, damage, etc. However, significant events, such as tornadoes, having no impact or causing no damage, are also included in Storm Data.


6. cz_type:              Indicates whether the event happened in   - C: County/Parish  - Z: NWS zone  - M: Marine


7. cz_fips_code:         Unique FIPS code identifier assigned to each county.   State names and their corresponding FIPS codes are available as a BigQuery Public Dataset: bigquery-public-data.census_fips_codes.counties_2016  The geographic polygons that define the perimeter of each state are available as a BigQuery Public Dataset: bigquery-public-data.geo_us_boundaries.us_counties.


8. cz_name:              (County/Parish, Zone or Marine Name assigned to the county FIPS number or NWS Forecast Zone  NWS Forecast Zones are available as a BigQuery Public Dataset: bigquery-public-data.noaa_historic_severe_storms.nws_forecast_zones.


9. wfo:                  National Weather Service Forecast Office’s area of responsibility (County Warning Area) in which the event occurred.


10. event_begin_time:     The date and time that the event began. Note that episodes and events may have different start and end times if multiple events occured in the same episode.


11. event_timezone:       The time zone in which the event_begin_time and the event_end_time is recorded.


12. event_end_time:       The date and time that the event ended. Note that episodes and events may have different start and end times if multiple events occured in the same episode.


13. injuries_direct:      The number of injuries directly related to the weather event.


14. injuries_indirect:    The number of injuries indirectly related to the weather event.


15. deaths_direct:        The number of deaths directly related to the weather event.


16. deaths_indirect:      The number of deaths indirectly related to the weather event.


17. damage_property:      The estimated amount of damage to property incurred by the weather event, in USD at the time of the event. Values are not adjusted for inflation  Note: Values listed as 0 do not necessarily mean that no property damage occurred as a result of the event.


18. damage_crops:         The estimated amount of damage to crops incurred by the weather event, in USD at the time of the storm. Values are not adjusted for inflation  Note: Values listed as 0 do not necessarily mean that no property damage occurred as a result of the event.


19. source:               Source reporting the weather event  Note: This can be any entry. Values are not restricted to specific categories.


20. magnitude:            Measured extent of the magnitude type. This is only used for wind speeds and hail size. Wind speeds are in MPH; Hail sizes are in inches.


21. magnitude_type:       Differentiates between the type of mangitude measured. - EG = Wind Estimated Gust  - ES = Estimated Sustained Wind  - MS = Measured Sustained Wind  - MG = Measured Wind Gust  No magnitude type is included for hail.


22. flood_cause:          Reported or estimated cause of the flood.


23. tor_f_scale:          Enhanced Fujita Scale describes the strength of the tornado based on the amount and type of damage caused by the tornado. The F-scale of damage will vary in the destruction area; therefore, the highest value of the F-scale is recorded for each event.    - EF0 – Light Damage (40 – 72 mph)   - EF1 – Moderate Damage (73 – 112 mph)   - EF2 – Significant damage (113 – 157 mph)   - EF3 – Severe Damage (158 – 206 mph)   - EF4 – Devastating Damage (207 – 260 mph)   - EF5 – Incredible Damage (261 – 318 mph).


24. tor_length:           Length of the tornado or tornado segment while on the ground (minimal of tenths of miles).


25. tor_width:            Width of the tornado or tornado segment while on the ground (in feet).


26. tor_other_wfo:        Indicates the continuation of a tornado segment as it crossed from one National Weather Service Forecast Office to another. The subsequent WFO identifier is provided within this field..


27. location_index:       Number assigned by NWS to specific locations within the same Storm event. Each event’s sequentially increasing location index number will have a corresponding lat/lon point.


28. event_range:          A hydro-meteorological event will be referenced, minimally, to the nearest tenth of a mile, to the geographical center (not from the village/city boundaries or limits) of a particular village/city, airport, or inland lake, providing that the reference point is documented in the Storm Data software location database.


29. event_azimuth:        16-point compass direction from a particular village/city, airport, or inland lake, providing that the reference point is documented in the Storm Data software location database of > 130,000 locations.


30. reference_location:   Reference location of the center from which the range is calculated and the azimuth is determined.


31. event_latitude:       The latitude where the event occurred (rounded to the hundredths in decimal degrees; includes an ‘-‘ if it’s S of the Equator).


32. event_longitude:      The longitude where the event occurred (rounded to the hundredths in decimal degrees; includes an ‘-‘ if it’s W of the Prime Meridian).


33. event_point:          Geographic representation of the event_longitude and latitude.

Αφαιρούμε τις παρακάτω στήλες, καθώς δε μας χρειάζονται στην ανάλυσή μας.

In [4]:
if True:   
    del df['source']
    del df['flood_cause']
    #del df['tor_length']
    #del df['tor_width']
    del df['tor_other_wfo']
    del df['location_index']
    del df['event_azimuth']
    del df['event_range']
    del df['reference_location']
    del df['cz_fips_code']
    del df['state_fips_code']
    del df['cz_name']
    del df['wfo']
    del df['event_timezone']
    
    del df['Unnamed: 0']
    # del df['episode_id']
    # del df['event_id']
    
    del df['event_point']
    # del df['magnitude_type']

Ενοποιήσαμε ορισμένες κατηγορίες φαινομένων και δημιουργήαμε νέες στήλες, όπως φαίνεται παρακάτω:

Generating new features:

- damage - > damage properties + damage crops
- injuries -> indirect + direct
- deaths   -> indirect + direct
- total_human_damage - > injuries + deaths

In [5]:
# Ενώνουμε τα deaths και τα injuries (direct - indirect) και μετά σβήνουμε τις άλλες (χωριστές) στήλες
df['deaths'] = df['deaths_direct'] + df['deaths_indirect']
df['injuries'] = df['injuries_direct'] + df['injuries_indirect']
# Δημιουργούμε μια καινούρια στήλη η οποία περιέχει το άθροισμα των deaths και των injuries
df['total_human_damage'] = df['injuries'] + df['deaths']
del df['injuries_direct']
del df['injuries_indirect']
del df['deaths_direct']
del df['deaths_indirect']

Εκτυπώνουμε τις μοναδικές τιμές των event types:

In [6]:
# unique event types
print('unique event types: ', len(df['event_type'].unique()))
print(df['event_type'].unique())

unique event types:  77
['Hail' 'Thunderstorm Wind' 'Tornado' 'THUNDERSTORM WINDS/ FLOOD'
 'THUNDERSTORM WINDS FUNNEL CLOU' 'HAIL FLOODING'
 'THUNDERSTORM WINDS/FLASH FLOOD' 'THUNDERSTORM WINDS HEAVY RAIN'
 'THUNDERSTORM WINDS/HEAVY RAIN' 'THUNDERSTORM WINDS LIGHTNING'
 'TORNADO/WATERSPOUT' 'THUNDERSTORM WIND/ TREE' 'THUNDERSTORM WIND/ TREES'
 'winter storm' 'winter weather' 'heavy snow' 'cold/wind chill'
 'high wind' 'strong wind' 'wildfire' 'thunderstorm wind' 'blizzard'
 'hail' 'tornado' 'flash flood' 'funnel cloud' 'heavy rain' 'heat'
 'lightning' 'dust devil' 'flood' 'extreme cold/wind chill' 'drought'
 'avalanche' 'coastal flood' 'high surf' 'marine thunderstorm wind'
 'marine strong wind' 'waterspout' 'lake-effect snow' 'ice storm'
 'frost/freeze' 'excessive heat' 'marine hail' 'tropical storm'
 'marine tropical storm' 'marine high wind' 'sleet' 'rip current'
 'tropical depression' 'storm surge/tide' 'sneakerwave' 'hurricane'
 'marine hurricane/typhoon' 'dense fog' 'lakeshore fl

Παρατηρηρούμε ότι μερικά από τα event types, όπως το hail που ήταν γραμμένο ως 'hail' αλλά και 'Hail', οπότε το διορθώσαμε μετατρέποντάς τα όλα lowercase.

In [7]:
df['event_type'] = df['event_type'].str.lower()
print(len(df['event_type'].unique()))

74


Ωστόσο, παρά τη διόρθωση αυτή, τα event types είναι πολλά. Επομένως, αποφασίσαμε αρχικά να συγχωνεύσουμε κάποια τα οποία μοιάζουν πολύ, όπως για παράδειγμα:

	thunderstorm_winds heavy rain και thunderstorm_winds / heavy rain → thunderstorm wind

In [8]:
# διορθώσεις ονομάτων στα event types
storm_types_mapping = {
'hail/icy roads': 'hail',
'hail flooding': 'hail',
'marine hail': 'hail',
'hail' : 'hail',
'thunderstorm winds funnel clou': 'thunderstorm wind',
'thunderstorm winds/flash flood': 'thunderstorm wind',
'thunderstorm winds heavy rain': 'thunderstorm wind',
'thunderstorm winds/heavy rain': 'thunderstorm wind',
'thunderstorm winds lightning': 'thunderstorm wind',
'thunderstorm winds/flooding': 'thunderstorm wind',
'thunderstorm winds/ flood': 'thunderstorm wind',
'marine thunderstorm wind': 'thunderstorm wind',
'thunderstorm wind/ trees': 'thunderstorm wind',
'thunderstorm wind/ tree': 'thunderstorm wind',
'thunderstorm wind': 'thunderstorm wind',
# https://oceanservice.noaa.gov/facts/cyclone.html   typhoon = hurricane
'hurricane (typhoon)': 'hurricane',
'marine hurricane/typhoon': 'hurricane',
'waterspout': 'hurricane',
'hurricane': 'hurricane',
'tornadoes, tstm wind, hail': 'tornado',
'tornado/waterspout': 'tornado',
'tornado': 'tornado',
'flash flood': 'flash flood',
'lakeshore flood': 'flood',
'coastal flood': 'flood',
'high surf': 'flood',
'flood': 'flood',
'drought': 'drought',
'heat': 'drought',
'cold/wind chill': 'cold/wind chill',
'extreme cold/wind chill': 'wind',
'marine strong wind': 'wind',
'strong wind': 'wind',
'heavy wind': 'wind',
'high wind': 'wind',
'lake-effect snow': 'snow',
'heavy snow': 'snow',
'avalanche': 'snow',
'high snow': 'snow',
'blizzard': 'snow',
'wildfire': 'wildfire',
'heavy rain': 'heavy rain',
'storm surge/tide': 'storm surge/tide',
'dust storm': 'storm',
'tsunami': 'tsunami',
'winter weather': 'winter weather',
'winter storm': 'winter storm',
'frost/freeze': 'frost/freeze',
'marine tropical depression': 'marine tropical depression',
'marine tropical storm': 'marine tropical storm',
'astronomical low tide': 'astronomical low tide',
'tropical depression': 'tropical depression',
'marine high wind': 'marine high wind',
'volcanic ashfall': 'volcanic ashfall',
'marine dense fog': 'marine dense fog',
'marine lightning': 'marine lightning',
'northern lights': 'northern lights',
'excessive heat': 'excessive heat',
'tropical storm': 'tropical storm',
'debris flow': 'debris flow',
'dense smoke': 'dense smoke',
'volcanic ash': 'volcanic ash',
'freezing fog': 'freezing fog',
'sneakerwave': 'sneakerwave',
'rip current': 'rip current',
'funnel cloud': 'funnel cloud',
'landslide': 'landslide',
'dust devil': 'dust devil',
'ice storm': 'ice storm',
'dense fog': 'dense fog',
'lightning': 'lightning',
'seiche': 'seiche',
'sleet': 'sleet',
'other': 'other'
}

def get_storm_type(k):
    return storm_types_mapping[k]

df.reset_index(drop=True)
df1 = copy.deepcopy(df)
df1['event_type'] = df1['event_type'].apply(get_storm_type)

print(df1.event_type.unique())
print(len(df1['event_type'].unique()))

['hail' 'thunderstorm wind' 'tornado' 'winter storm' 'winter weather'
 'snow' 'cold/wind chill' 'wind' 'wildfire' 'flash flood' 'funnel cloud'
 'heavy rain' 'drought' 'lightning' 'dust devil' 'flood' 'hurricane'
 'ice storm' 'frost/freeze' 'excessive heat' 'tropical storm'
 'marine tropical storm' 'marine high wind' 'sleet' 'rip current'
 'tropical depression' 'storm surge/tide' 'sneakerwave' 'dense fog'
 'freezing fog' 'storm' 'debris flow' 'astronomical low tide'
 'marine tropical depression' 'dense smoke' 'seiche' 'volcanic ash'
 'landslide' 'volcanic ashfall' 'marine dense fog' 'tsunami' 'other'
 'northern lights' 'marine lightning']
44


Τα είδη των φαινομένων εξακολουθούν να είναι πολλά (42), επομένως μελετάμε τη συχνότητα εμφάνισης του κάθε φαινομένου στο dataset:

In [9]:
temp = (df1['event_type'].value_counts() / len(df)) * 100
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp)

thunderstorm wind             29.740318
hail                          23.202018
wind                           6.465939
flash flood                    5.347943
snow                           4.972044
drought                        4.826285
winter storm                   4.654229
tornado                        4.370154
flood                          4.209264
winter weather                 3.656794
heavy rain                     1.547769
lightning                      1.068515
cold/wind chill                0.872054
dense fog                      0.821169
frost/freeze                   0.726539
ice storm                      0.713299
funnel cloud                   0.537033
excessive heat                 0.515740
wildfire                       0.455093
hurricane                      0.445331
tropical storm                 0.321964
rip current                    0.084258
storm surge/tide               0.082245
storm                          0.070226
debris flow                    0.068639


Αποφασίζουμε τελικά να αφαιρέσουμε αυτά που παρουσιάζουν τη πολύ μικρή συχνότητα εμφανίσεων. Συγκεκριμένα θα αφαιρέσουμε αυτά τα οποία έχουν ποσοστό εμφάνισης όλα αυτά τα χρόνια <0.01% επί του συνόλου.

In [10]:
temp = temp[temp < 0.01]
event_types_we_dont_keep = []
for i in temp.index:
    event_types_we_dont_keep.append(i)
print('Event types we dont keep:\n', event_types_we_dont_keep)
print("")
df1.reset_index(drop=True)
df2 = df1[~df1['event_type'].isin(event_types_we_dont_keep)]
print('Event types we keep:\n',df2['event_type'].unique())

print(len(df2['event_type'].unique()))

Event types we dont keep:
 ['marine tropical storm', 'dense smoke', 'volcanic ash', 'volcanic ashfall', 'seiche', 'tsunami', 'sneakerwave', 'northern lights', 'marine dense fog', 'marine tropical depression', 'other', 'marine lightning']

Event types we keep:
 ['hail' 'thunderstorm wind' 'tornado' 'winter storm' 'winter weather'
 'snow' 'cold/wind chill' 'wind' 'wildfire' 'flash flood' 'funnel cloud'
 'heavy rain' 'drought' 'lightning' 'dust devil' 'flood' 'hurricane'
 'ice storm' 'frost/freeze' 'excessive heat' 'tropical storm'
 'marine high wind' 'sleet' 'rip current' 'tropical depression'
 'storm surge/tide' 'dense fog' 'freezing fog' 'storm' 'debris flow'
 'astronomical low tide' 'landslide']
32


Για τα χαρακτηριστικά event_latitude , event_longitude, τα οποία αναφέρονται σε συντεταγμένες, κάνουμε την εξής επεξεργασία: εφόσον, οι περιοχές είναι πάντα στις ίδιες συντεταγμένες, πήραμε το μέσο όρο του long και του lat για κάθε state και αντικαθιστούμε τα nan values στις γεωγραφικές τιμές με αυτά.

In [11]:
long_lat_mean = df2.groupby('state')['event_longitude','event_latitude'].sum() / df2.groupby('state')['event_longitude','event_latitude'].count()
long_lat_mean

,event_longitude,event_latitude
state,,
Alabama,-86.794329,33.273120
Alaska,-148.691622,62.656039
American samoa,-170.606827,-9.855073
Arizona,-111.818135,33.593247
Arkansas,-92.645165,34.977785
...,...,...
Virginia,-78.658510,37.610194
Washington,-119.355004,47.343790
West virginia,-80.683010,38.781910


In [12]:
# this is for replacing all values with the mean of their lat,long
df2.reset_index(drop=True)
df3 = copy.deepcopy(df2)

for i in df3.state.unique()[:-1]: 
    df3.loc[df['state'] == i, 'event_latitude'] = long_lat_mean.loc[i]['event_latitude']
    df3.loc[df['state'] == i, 'event_longitude'] = long_lat_mean.loc[i]['event_longitude']
    
df3 = df3[df3['event_latitude'].notna()]
df3 = df3[df3['event_longitude'].notna()]

In [13]:
df4 = copy.deepcopy(df3)
df4['event_longitude'] = pd.to_numeric(df4['event_longitude'])
df4['event_latitude'] = pd.to_numeric(df4['event_latitude'])

Επίσης, αντικαθιστούμε τα nan values των στηλών damage_property, damage_crops, deaths και injuries με 0.

In [14]:
df4.reset_index(drop=True)
df5 = copy.deepcopy(df4)
### setting nan values of damage properties to 0.
df5['damage_property'] = df5['damage_property'].replace(np.nan, 0)
### setting nan values of damage corps to 0.
df5['damage_crops'] = df5['damage_crops'].replace(np.nan, 0)
### setting nan values of deaths to 0.
df5['deaths'] = df5['deaths'].replace(np.nan, 0)   
### setting nan values of injuries to 0.
df5['injuries'] = df5['injuries'].replace(np.nan, 0)

Όπως είδαμε, εκτός απο γεωγραφικά δεδομένα, έχουμε και χρονικά δεδομένα. Επομένως, τα στατιστικά τα οποία αναφέρονται σε USD(us dollars) πρέπει να μετατραπούν ανάλογα με τον πληθωρισμό.

Κατεβάσαμε, επομένως, το dataset για το [CPI(Consumer Price Index)](https://fred.stlouisfed.org/series/CPIAUCSL). Μια μεθοδολογία για το πως μετατρέπουμε τις τιμές βάσει της χρονολογίας που θέλουμε βρίσκεται [εδώ]((https://www.rba.gov.au/education/resources/explainers/inflation-and-its-measurement.html))

Φορτώνουμε το αρχείο με τα δεδομένα για τον πληθωρισμό και στη συνέχεια δημιουργούμε μια νέα στήλη με τον πολλαπλασιαστή για κάθε χρονιά. Στη δική μας περίπτωση μετατρέπουμε τα δεδομένα μας με βάση το 2019.

In [15]:
# load inflation data
inflation = pd.read_csv('inflation_data.csv')
# create index multiplier
inflation['CPI_Multiplier'] = inflation['CPI'].iloc[-1] / inflation['CPI']

Μετατρέπουμε τη στήλη DATE η οποία έχει αναλυτικά της ημερομηνίες(1994-12-1) και κρατάμε μόνο τη χρονιά(1994). Στη συνέχεια, κρατάω τις χρονιές από το 1950 και μετά, καθώς αυτές μας ενδιαφέρουν. Τέλος, βρίσκω το μέσο όρο για κάθε χρονιά και αυτή η τιμή θα αντιπροσωπεύει το CPI MULTIPLIER για τη χρονιά εκείνη(καθώς τα δεδομένα μας έχουν για κάθε μέρα ξεχωριστά και εμείς θα το προσεγγίσουμε ανά χρονιά).

In [ ]:
inflation ['DATE'] = inflation ['DATE'].apply(lambda x: int(x.split('-')[0]))
inflation = inflation[inflation['DATE'] >=1950]
del inflation['CPI']
my_inflation = inflation.groupby('DATE').sum() / inflation.groupby('DATE').count()

# creating a DATE column same as inflation on our storm dataset so that i can
# relate inflation and storm dataset
df5.reset_index(drop=True)
df6 = copy.deepcopy(df5)
df6['DATE'] = df6['event_begin_time'].apply(lambda x: x.split('-')[0])
df6['DATE'] = df6['DATE'].apply(lambda x: int(x))

# fix inflation with base 2019
for i in df6.DATE.unique():
    df6.loc[df6['DATE'] == i,['damage_property']] *= my_inflation.loc[i].values[0]
    df6.loc[df6['DATE'] == i,['damage_crops']] *= my_inflation.loc[i].values[0]

Επιπλέον, είδαμε τις εξής πληροφορίες για τις στήλες 'Magnitude', 'Magnitude type' και 'tor_f_scale':

'Magnitude' : only used for wind speeds and hail size 
              Wind speeds are in MPH; Hail sizes are in inches

'Magnitude type' : Differentiates between the type of mangitude measured. No magnitude type included for hail.
- EG = Wind Estimated Gust 
- ES = Estimated Sustained Wind  
- MS = Measured Sustained Wind  
- MG = Measured Wind Gust 

'tor_f_scale': "Enhanced Fujita Scale" describes the strength of the tornado based on the amount and type of damage caused by the tornado. The F-scale of damage will vary in the destruction area; therefore, the highest value of the F-scale is recorded for each event.   
- EF0 – Light Damage (40 – 72 mph)  
- EF1 – Moderate Damage (73 – 112 mph)  
- EF2 – Significant damage (113 – 157 mph)
- EF3 – Severe Damage (158 – 206 mph)  
- EF4 – Devastating Damage (207 – 260 mph)  
- EF5 – Incredible Damage (261 – 318 mph)

Επίσης, η τιμή 'EFU' που θα δούμε ότι υπάρχει είναι αντίστοιχο του 'unknown'.
https://en.wikipedia.org/wiki/Enhanced_Fujita_scale

In [ ]:
df6['tor_f_scale'].unique()

Βλέπουμε ότι το χαρακτηριστικό tor_f_scale παρουσιάζει και αυτό μια ασυνέπεια στα δεδομένα του. Αυτό, γιατί θα έπρεπε να αποτελείται από τις τιμές EF0,EF1...EF5 αλλά υπάρχουν και τιμές F0,F1..,F5. Επομένως τα μετατρέπουμε όλα στην ίδια τιμή.

In [ ]:
# inconsistency των δεδομένων και αντίστοιχες διορθώσεις

df6['tor_f_scale'].replace('F0', 'EF0', inplace=True)
df6['tor_f_scale'].replace('F1', 'EF1', inplace=True)
df6['tor_f_scale'].replace('F2', 'EF2',inplace=True)
df6['tor_f_scale'].replace('F3', 'EF3',inplace=True)
df6['tor_f_scale'].replace('F4', 'EF4',inplace=True)
df6['tor_f_scale'].replace('F5', 'EF5',inplace=True)
df6['tor_f_scale'].replace('F6', 'EF6',inplace=True)
df6['tor_f_scale'].replace('EFU', 'NaN',inplace=True)
df6['tor_f_scale'].fillna('NaN',inplace=True)

df6['tor_f_scale'].replace('EF0', 0, inplace=True)
df6['tor_f_scale'].replace('EF1', 1, inplace=True)
df6['tor_f_scale'].replace('EF2', 2, inplace=True)
df6['tor_f_scale'].replace('EF3', 3, inplace=True)
df6['tor_f_scale'].replace('EF4', 4, inplace=True)
df6['tor_f_scale'].replace('EF5', 5, inplace=True)

df6['tor_f_scale'].unique()

In [ ]:
# διορθώνουμε το index
df6.reset_index(drop=True)
df7 = copy.deepcopy(df6)
df7['index'] = np.arange(len(df7))
df7.set_index('index', inplace=True)
print('columns:\n')
for i in df7.columns: print(i)
print('\n')
print('Dataframe after preprocessing:\n')
df7.head(5)

In [ ]:
df7.to_csv(r'preprocessed_data.csv')